In [5]:
from google.colab import drive
drive.mount('/content/drive/')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive/


# **Training Face Mask Detection Model**

In [6]:
!pip install -q keras

In [7]:
import keras

Using TensorFlow backend.


In [8]:
from tensorflow.keras.preprocessing.image import img_to_array, load_img, ImageDataGenerator
from tensorflow.keras.layers import Input, Lambda, Dense, Flatten, Dropout
from tensorflow.keras.models import Model
from keras.models import Sequential
import numpy as np
from glob import glob
import matplotlib.pyplot as plt
%matplotlib inline
import argparse
import os
from imutils import paths

IMAGE_SIZE = [224,224]

test_path = '/content/drive/My Drive/Colab Notebooks/cell/Test'
train_path = '/content/drive/My Drive/Colab Notebooks/cell/Train'


In [9]:
import tensorflow

In [10]:
#setting the images into same size 
train_datagen = ImageDataGenerator(rescale = 1./255,shear_range=0.2,zoom_range=0.2,horizontal_flip=True)

In [11]:
test_datagen = ImageDataGenerator(rescale = 1./255)

In [12]:
training_set = train_datagen.flow_from_directory('/content/drive/My Drive/Colab Notebooks/cell/Train',target_size=(244,244),batch_size=32,class_mode='categorical')

Found 3989 images belonging to 2 classes.


In [13]:
test_set = test_datagen.flow_from_directory('/content/drive/My Drive/Colab Notebooks/cell/Test',target_size=(244,244),batch_size=32,class_mode='categorical')

Found 977 images belonging to 2 classes.


In [14]:
trainImages = list(paths.list_images('/content/drive/My Drive/Colab Notebooks/cell/Train'))

In [15]:
testImages = list(paths.list_images('/content/drive/My Drive/Colab Notebooks/cell/Test'))

In [16]:
#importing images into file
from tensorflow.keras.applications.mobilenet_v2 import preprocess_input
train = []
labels = []
for i in trainImages: 
  label = i.split(os.path.sep)[-2]
  img = load_img(i, target_size = (244, 244), interpolation = "nearest", color_mode = "rgb")
  img = img_to_array(img)
  img = preprocess_input(img)
  train.append(img)
  labels.append(label)


/usr/local/lib/python3.6/dist-packages/PIL/Image.py:932: UserWarning: Palette images with Transparency expressed in bytes should be converted to RGBA images
  "Palette images with Transparency expressed in bytes should be "


In [17]:
#Loaing the images into numpy array
train = np.array(train)
labels = np.array(labels)

In [18]:
#Converting labels numpy array as a categorical data
from sklearn.preprocessing import LabelBinarizer
from keras.utils.np_utils import to_categorical   
lb = LabelBinarizer()
labels = lb.fit_transform(labels)
labels = to_categorical(labels)

In [19]:
#Spliting the data into train and test datasets
from sklearn.model_selection import train_test_split
X_Train, X_Test, y_train, y_test = train_test_split(train, labels, train_size = 0.75, random_state = 40, stratify = labels)

In [20]:
print(X_Train.shape, X_Test.shape, y_train.shape, y_test.shape)

(2991, 244, 244, 3) (998, 244, 244, 3) (2991, 2) (998, 2)


In [21]:
#Image Data Generator is used to generate different images from the train data while passing multiple times to model
IDG = ImageDataGenerator(rotation_range = 50, width_shift_range = 0.5, height_shift_range = 0.5, horizontal_flip = True, vertical_flip = True, fill_mode = 'nearest')

In [22]:
#Input is passed to Mobilevnet2 algorithm to train the model
from tensorflow.keras.applications import MobileNetV2
base = MobileNetV2(alpha = 1.0, include_top = False, weights = 'imagenet',input_tensor = Input(shape = (224, 224, 3)), classes = 2)

9412608/9406464 [==============================] - 0s 0us/step


In [23]:
#Second layer of the model is being given layers to train
finalModel = base.output
from tensorflow.keras.layers import AveragePooling2D, Dropout, Flatten, Dense
finalModel = AveragePooling2D(pool_size=(7, 7))(finalModel)
finalModel = Flatten(name = "flatten")(finalModel)
finalModel = Dense(128, activation = "relu")(finalModel)
finalModel = Dropout(0.5)(finalModel)
finalModel = Dense(2, activation = "softmax")(finalModel)

In [24]:
#Adding first layer and second layer of the models
model = Model(inputs = base.input, outputs = finalModel)

In [25]:
#Making the base layers as non trainable
for layers in base.layers:
  layers.trainable = False

In [26]:
#Setting learning rate, epochs (the number of times the data is passed to model), and batch size
INIT_LR = 1e-4
EPOCHS = 40
BS = 32
#Adamax is a optimizer which is used to optimize the model
from tensorflow.keras.optimizers import Adamax
model.compile(loss = "categorical_crossentropy", optimizer = Adamax(lr = INIT_LR, decay = INIT_LR // EPOCHS), metrics = ["accuracy"])


In [27]:
#Fitting the main model on the train data
main = model.fit(IDG.flow(X_Train, y_train, batch_size = BS), validation_data = (X_Test, y_test), validation_steps = len(X_Test) // BS, epochs = EPOCHS, steps_per_epoch = len(X_Train) // BS)

Epoch 1/40
93/93 [==============================] - 181s 2s/step - loss: 0.7778 - accuracy: 0.5789 - val_loss: 0.5165 - val_accuracy: 0.7625
Epoch 2/40
93/93 [==============================] - 181s 2s/step - loss: 0.5162 - accuracy: 0.7374 - val_loss: 0.3583 - val_accuracy: 0.8818
Epoch 3/40
93/93 [==============================] - 182s 2s/step - loss: 0.4198 - accuracy: 0.8080 - val_loss: 0.2801 - val_accuracy: 0.9158
Epoch 4/40
93/93 [==============================] - 182s 2s/step - loss: 0.3754 - accuracy: 0.8432 - val_loss: 0.2222 - val_accuracy: 0.9429
Epoch 5/40
93/93 [==============================] - 182s 2s/step - loss: 0.3407 - accuracy: 0.8510 - val_loss: 0.1880 - val_accuracy: 0.9499
Epoch 6/40
93/93 [==============================] - 182s 2s/step - loss: 0.3395 - accuracy: 0.8614 - val_loss: 0.1671 - val_accuracy: 0.9589
Epoch 7/40
93/93 [==============================] - 180s 2s/step - loss: 0.2976 - accuracy: 0.8814 - val_loss: 0.1559 - val_accuracy: 0.9609
Epoch 8/40
93

In [28]:
#Predicting the output on test data from the trained model
y_pred = model.predict(x = X_Test, batch_size = BS)

In [29]:
#Setting the value as maximum that is depending on the percentage of mask present predicting it as mask present or not present
y_pred = np.argmax(y_pred, axis=1)

In [30]:
#Setting the value as maximum that is depending on the percentage of mask present predicting it as mask present or not present
y_test = y_test.argmax(axis = 1)

In [31]:
#Predicting accuracy
from sklearn.metrics import r2_score
r_squared = r2_score(y_test, y_pred)

In [32]:
print(r_squared)

0.9078152610441766


In [33]:
#Saving the model
model.save('finalversion.h5')